In [1]:
#!gsutil -m cp -r gs://the_impostor_hunt/data ./  

In [2]:
# ler o arquivo data/train.csv
# para cada id na tabela train_roles no formato com 4 numeros (0001, 0002, 0010)
# ler cada um dos arquivos na pasta data/train/pasta

In [3]:
import pandas as pd
import json

In [4]:
train_rules = pd.read_csv("data/train.csv")

In [5]:
train_rules.head()

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2


In [ ]:

for i in train_rules.values:
    text_id = i[1]
    if text_id == 1:
        text_id_test = 2
    else:
        text_id_test = 1
    #print(indice, text_id)
    id_formatado_train.append(indice)

In [6]:
id_formatado_train = []
for i in train_rules.values:
    indice = str(i[0]).zfill(4)  # converte para string e preenche com zeros até ter 4 dígitos
    text_id = i[1]
    #print(indice, text_id)
    id_formatado_train.append(indice)

In [7]:
id_formatado_train[:5]

['0000', '0001', '0002', '0003', '0004']

In [8]:
train_rules['id_formatado_train'] = id_formatado_train

In [9]:
train_rules.head()

,id,real_text_id,id_formatado_train
0,0,1,0000
1,1,2,0001
2,2,1,0002
3,3,2,0003
4,4,2,0004


In [10]:
train_file = []
for i in train_rules.values:
    indice = str(i[0]).zfill(4)  # converte para string e preenche com zeros até ter 4 dígitos
    train_id = i[1]
    id_formatado = i[2]
    train_file.append(f"data/train/article_{id_formatado}/file_{train_id}.txt")
train_rules['train_file'] = train_file

In [11]:
train_rules.head()

,id,real_text_id,id_formatado_train,train_file
0,0,1,0000,data/train/article_0000/file_1.txt
1,1,2,0001,data/train/article_0001/file_2.txt
2,2,1,0002,data/train/article_0002/file_1.txt
3,3,2,0003,data/train/article_0003/file_2.txt
4,4,2,0004,data/train/article_0004/file_2.txt


In [13]:
data_list = []

for i in train_rules.values:
    file_path_train = i[3]  # caminho do arquivo
    print(file_path_train)
    
    # Lê o conteúdo do arquivo
    with open(file_path_train, 'r', encoding='utf-8') as f:
        text = f.read().strip()
    
    # Adiciona ao dicionário
    data_list.append({
        "file_train": file_path_train,
        "train": text
    })

# Salva como JSON
with open("train.json", "w", encoding="utf-8") as f:
    json.dump(data_list, f, ensure_ascii=False, indent=2)

data/train/article_0000/file_1.txt
data/train/article_0001/file_2.txt
data/train/article_0002/file_1.txt
data/train/article_0003/file_2.txt
data/train/article_0004/file_2.txt
data/train/article_0005/file_1.txt
data/train/article_0006/file_1.txt
data/train/article_0007/file_1.txt
data/train/article_0008/file_1.txt
data/train/article_0009/file_2.txt
data/train/article_0010/file_1.txt
data/train/article_0011/file_1.txt
data/train/article_0012/file_1.txt
data/train/article_0013/file_1.txt
data/train/article_0014/file_2.txt
data/train/article_0015/file_2.txt
data/train/article_0016/file_1.txt
data/train/article_0017/file_1.txt
data/train/article_0018/file_2.txt
data/train/article_0019/file_2.txt
data/train/article_0020/file_1.txt
data/train/article_0021/file_2.txt
data/train/article_0022/file_1.txt
data/train/article_0023/file_1.txt
data/train/article_0024/file_2.txt
data/train/article_0025/file_1.txt
data/train/article_0026/file_1.txt
data/train/article_0027/file_2.txt
data/train/article_0

In [14]:
# Converte para DataFrame
df = pd.DataFrame(data_list)

# Visualiza o DataFrame
df.head()

,file_train,train
0,data/train/article_0000/file_1.txt,The VIRSA (Visible Infrared Survey Telescope A...
1,data/train/article_0001/file_2.txt,The project aims to achieve an accuracy level ...
2,data/train/article_0002/file_1.txt,Scientists can learn about how galaxies form a...
3,data/train/article_0003/file_2.txt,The importance for understanding how stars evo...
4,data/train/article_0004/file_2.txt,Analyzing how fast stars rotate within a galax...
